In [12]:
using GAP
using JSON3


In [13]:
# json ファイルの読み取り
config_raw = JSON3.read(read("config.json", String))

"""
JSONのサイクルリストからGAPの置換オブジェクトを生成する
例: [[1,3,8,6], [2,5,7,4]]  =>  (1,3,8,6)(2,5,7,4)
"""
function reconstruct_gap_perm(cycles_list)
    # 各サイクルを "(1,3,8,6)" のような文字列に変換し、結合する
    # GAP.evalstr は文字列をGAPの式として評価してオブジェクトを返します
    gap_str = join(["(" * join(c, ",") * ")" for c in cycles_list], "")

    # 恒等置換（どこも動かない）の場合は空文字列になるので、その場合は "()" を渡す
    if gap_str == ""
        return @gap ()
    end

    return GAP.evalstr(gap_str)
end


reconstruct_gap_perm

In [14]:
# 2. 各面を再構築して辞書に格納
# これで Core 定義を外部ファイルから注入できます
moves = Dict()
for (face_name, cycles) in config_raw.definitions
    moves[string(face_name)] = reconstruct_gap_perm(cycles)
end


In [15]:
# 3. 以前の Core 定義のように変数へ割り当てる
U, L, F, R, B, D = moves["U"], moves["L"], moves["F"], moves["R"], moves["B"], moves["D"]

cube = (@gap Group)(U, L, F, R, B, D)
words = @gap ["U", "L", "F", "R", "B", "D"]
free = (@gap FreeGroup)(words)
hom = (@gap GroupHomomorphismByImages)(free, cube, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube))

scramble_array = [rand(["U", "L", "F", "R", "B", "D"]) for _ in 1:20]
scramble_str = join(scramble_array, " ")

σ = @gap ()
for s in scramble_array
    idx = findfirst(==(s), ["U", "L", "F", "R", "B", "D"])
    global σ *= (@gap GeneratorsOfGroup)(cube)[idx]
end

word_raw = string((@gap PreImagesRepresentative)(hom, σ))

function gap_to_lsystem(word_str)
    res = replace(word_str, "*" => " ")
    res = replace(res, "^-1" => "-")
    res = replace(res, "^-2" => "++")
    res = replace(res, "^2" => "++")
    return join([occursin(r"[\+\-]", m) ? m : m * "+" for m in split(res)], " ")
end


gap_to_lsystem (generic function with 1 method)

In [26]:
"""
現在のステッカー配置（current_state）に、
指定された置換（p_gap: GAPのオブジェクト）を適用して新しい配置を返す
"""
function apply_move_to_state(current_state::Vector{Int}, p_gap)
    # 1. Juliaのループで各番号の移動先を計算する
    # GAP.jlにより、整数 ^ GapObj (置換) で移動後の値が返ります
    p_map = [Int(i^p_gap) for i in 1:48]
    
    # 2. 新しい状態を作る（物理的な移動）
    next_state = copy(current_state)
    for i in 1:48
        next_state[p_map[i]] = current_state[i]
    end
    
    return next_state
end

# 動作確認
current_state = collect(1:48)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, F)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, U)
current_state = apply_move_to_state(current_state, U)
println(current_state)


[1, 2, 3, 4, 5, 16, 13, 11, 9, 10, 41, 12, 42, 14, 15, 43, 22, 20, 17, 23, 18, 24, 21, 19, 6, 26, 27, 7, 29, 8, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 30, 28, 25, 44, 45, 46, 47, 48]


In [27]:
data = Dict(
    "current" => current_state,
    "scramble" => gap_to_lsystem(scramble_str),
    "solution" => gap_to_lsystem(word_raw)
)
open("../src/config/solve.json", "w") do f
    JSON3.pretty(f, data)
end
